In [1]:
from mako.template import Template

import argparse
import configparser
from itertools import chain
import re

from sys import platform as _platform
import os

In [2]:
DEBUG = True
IMAGE_NAME = 'netkit'
LINUX_TERMINAL_TYPE = 'xterm'
PATH_TO_LAB = 'shared/MARACAS_lab/'

In [3]:
if _platform == "darwin":
    platform = 'MAC OS X'
elif _platform == "win32":
    platform = 'Windows'
else: #if _platform == "linux" or _platform == "linux2":
    platform = 'linux'
    
print platform

Windows


# Parsing of lab.conf

In [4]:
config = configparser.ConfigParser()

In [5]:
# reads lab.conf
with open(PATH_TO_LAB + 'lab.conf') as stream:
    # adds a section to mimic a .ini file
    stream = chain(("[dummysection]",), stream)
    config.read_file(stream)

In [6]:
# gets 2 list of keys, one for machines and the other for the metadata
# we also need a unique list of links
keys = []
m_keys = []
links = []
for key in config['dummysection']: 
    if DEBUG: print(key, config['dummysection'][key])
    if '[' in key and ']' in key:
        keys.append(key)
        links.append(config['dummysection'][key])
    else:
        m_keys.append(key)

('lab_description', u'"absaaaaaaaaaaaaaaaaa"')
('lab_version', u'"1.0"')
('lab_author', u'"aaaaaaaaaaaaaaaa"')
('lab_email', u'"aaaaaaaaa@aaaaaaaaaaaaaa"')
('host[0]', u'A')
('server[0]', u'B')
('ospf1[0]', u'C')
('ospf1[1]', u'E')
('ospf1[2]', u'F')
('ospf2[0]', u'D')
('ospf2[1]', u'E')
('ospf3[0]', u'A')
('ospf3[1]', u'D')
('ospf3[2]', u'C')
('br[0]', u'G')
('br[1]', u'F')
('rip1[0]', u'H')
('rip1[1]', u'G')
('rip2[0]', u'B')
('rip2[1]', u'H')


In [7]:
config['dummysection'][keys[0]]

u'A'

In [8]:
# we only need unique links
links = set(links)

In [9]:
links

{u'A', u'B', u'C', u'D', u'E', u'F', u'G', u'H'}

In [10]:
# helper functions for natural sorting
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

# sort the keys so that we keep the order of the interfaces
keys.sort(key=natural_keys)

In [11]:
keys

['br[0]',
 'br[1]',
 'host[0]',
 'ospf1[0]',
 'ospf1[1]',
 'ospf1[2]',
 'ospf2[0]',
 'ospf2[1]',
 'ospf3[0]',
 'ospf3[1]',
 'ospf3[2]',
 'rip1[0]',
 'rip1[1]',
 'rip2[0]',
 'rip2[1]',
 'server[0]']

In [12]:
m_keys

['lab_description', 'lab_version', 'lab_author', 'lab_email']

In [13]:
# we then get a dictionary of machines ignoring interfaces that have missing positions (eg: 1,3,6 instead of 0,1,2)
machines = {}
for key in keys: 
    splitted = key.split('[')
    name = splitted[0]
    splitted = splitted[1].split(']')
    ifnumber = int(splitted[0])
    if not machines.get(name):
        machines[name] = []
    if len(machines[name]) == 0 or machines[name][len(machines[name])-1][1] == ifnumber - 1:
        machines[name].append((config['dummysection'][key], ifnumber))

In [14]:
machines

{'br': [(u'G', 0), (u'F', 1)],
 'host': [(u'A', 0)],
 'ospf1': [(u'C', 0), (u'E', 1), (u'F', 2)],
 'ospf2': [(u'D', 0), (u'E', 1)],
 'ospf3': [(u'A', 0), (u'D', 1), (u'C', 2)],
 'rip1': [(u'H', 0), (u'G', 1)],
 'rip2': [(u'B', 0), (u'H', 1)],
 'server': [(u'B', 0)]}

In [15]:
metadata = {}
for m_key in m_keys:
    if config['dummysection'][m_key].startswith('"') and config['dummysection'][m_key].endswith('"'):
        config['dummysection'][m_key] = config['dummysection'][m_key][1:-1]
    metadata[m_key] = config['dummysection'][m_key]

In [16]:
metadata

{'lab_author': u'aaaaaaaaaaaaaaaa',
 'lab_description': u'absaaaaaaaaaaaaaaaaa',
 'lab_email': u'aaaaaaaaa@aaaaaaaaaaaaaa',
 'lab_version': u'1.0'}

# Generation of docker-compose.yml (may not be used in the end)

In [17]:
network_block_template = '''networks: 
${links}
'''
network_link_template = '''    ${link}:
'''
service_block_template = '''version: '2.1'

services:
${services}
'''
service_template = '''    ${machine_name}:
        image: ${image_name}
        privileged: true
        container_name: netkit-${machine_name}
        ports:
            - 808${number}:80
        networks:
${links}
'''
service_link_template = '''            ${link}:
'''
# TODO lab.dep with '''depends_on: service: condition: service_healty'''

In [18]:
print(Template('''hello ${data}! 
               ''').render(data="world"))

hello world! 
               


In [19]:
conf_network = ""
for link in links:
    conf_network = conf_network + Template(network_link_template).render(link=link)

conf_network = Template(network_block_template).render(links=conf_network)

In [20]:
print(conf_network)

networks: 
    A:
    C:
    B:
    E:
    D:
    G:
    F:
    H:




In [21]:
conf_services = ""
counter = 0
for machine_name, interfaces in machines.items():
    conf_links = ""
    for link, _ in interfaces:
        conf_links = conf_links + Template(service_link_template).render(link=link)
    conf_services = conf_services + Template(service_template).render(machine_name=machine_name, image_name=IMAGE_NAME, number=counter, links=conf_links)
    counter += 1
    
conf_services = Template(service_block_template).render(services=conf_services)

In [22]:
print(conf_services)

version: '2.1'

services:
    host:
        image: quagga
        privileged: true
        container_name: netkit-host
        ports:
            - 8080:80
        networks:
            A:

    server:
        image: quagga
        privileged: true
        container_name: netkit-server
        ports:
            - 8081:80
        networks:
            B:

    rip2:
        image: quagga
        privileged: true
        container_name: netkit-rip2
        ports:
            - 8082:80
        networks:
            B:
            H:

    rip1:
        image: quagga
        privileged: true
        container_name: netkit-rip1
        ports:
            - 8083:80
        networks:
            H:
            G:

    br:
        image: quagga
        privileged: true
        container_name: netkit-br
        ports:
            - 8084:80
        networks:
            G:
            F:

    ospf1:
        image: quagga
        privileged: true
        container_name: netkit-ospf1
        ports:

# Generation of docker run and docker network commands (lstart)

In [23]:
create_network_template = 'docker network create '
create_network_commands = []
for link in links:
    create_network_commands.append(create_network_template + link)

In [24]:
if platform != 'Windows':
    docker = 'docker'
else: 
    docker = 'sudo docker'

create_machine_template = docker + ' run -d --privileged --name {machine_name} -p --hostname={machine_name} 808{number}:80 --network={first_link} {image_name}'
# we could use -ti -a stdin -a stdout and then /bin/bash -c "commands;bash", 
# but that woult execute commands like ifconfig BEFORE all the networks are linked
create_machine_commands = []

create_connection_template = docker + ' network connect {link} {machine_name}'
create_connection_commands = []

copy_folder_template = docker + ' cp ' + PATH_TO_LAB + '{machine_name}/etc {machine_name}:/'
copy_folder_commands = []

exec_template = docker + ' exec {params} -ti --privileged=true {machine_name} {command}'
exec_commands = []

count = 0
def replace_multiple_items(repls, string):
    return reduce(lambda a, kv: a.replace(*kv), repls, string)

for machine_name, interfaces in machines.items():
    repls = ('{machine_name}', machine_name), ('{number}', str(count)), ('{first_link}', interfaces[0][0]), ('{image_name}', IMAGE_NAME)
    create_machine_commands.append(replace_multiple_items(repls, create_machine_template))
    count += 1
    for link,_ in interfaces[1:]:
        repls = ('{link}', link), ('{machine_name}', machine_name)
        create_connection_commands.append(replace_multiple_items(repls, create_connection_template))
    repls = ('{machine_name}', machine_name), ('{machine_name}', machine_name)
    copy_folder_commands.append(replace_multiple_items(repls, copy_folder_template))
    repls = ('{machine_name}', machine_name), ('{command}', 'bash  -c "echo -ne \'\\033]0;' + machine_name + '\\007\'; bash"'), ('{params}', '-e TERM=debian')
    exec_commands.append(replace_multiple_items(repls, exec_template))
    
    
# for each machine we have to get the machine.startup file and insert every non empty line as a string inside an array
startup_commands = []
for machine_name, _ in machines.items():
    f = open(PATH_TO_LAB + machine_name + '.startup', 'r')
    for line in f:
        if line.strip() and line not in ['\n', '\r\n']:
            repls = ('{machine_name}', machine_name), ('{command}', line.strip()), ('{params}', '-d')
            startup_commands.append(replace_multiple_items(repls, exec_template))
    f.close()

In [25]:
print create_network_commands
print create_machine_commands
print create_connection_commands
print copy_folder_commands

[u'docker network create A', u'docker network create C', u'docker network create B', u'docker network create E', u'docker network create D', u'docker network create G', u'docker network create F', u'docker network create H']
[u'docker run -d --privileged --name host -p 8080:80 --network=A quagga', u'docker run -d --privileged --name server -p 8081:80 --network=B quagga', u'docker run -d --privileged --name rip2 -p 8082:80 --network=B quagga', u'docker run -d --privileged --name rip1 -p 8083:80 --network=H quagga', u'docker run -d --privileged --name br -p 8084:80 --network=G quagga', u'docker run -d --privileged --name ospf1 -p 8085:80 --network=C quagga', u'docker run -d --privileged --name ospf3 -p 8086:80 --network=A quagga', u'docker run -d --privileged --name ospf2 -p 8087:80 --network=D quagga']
[u'docker network connect H rip2', u'docker network connect G rip1', u'docker network connect F br', u'docker network connect E ospf1', u'docker network connect F ospf1', u'docker network

In [26]:
commands = create_network_commands + create_machine_commands + create_connection_commands + copy_folder_commands

In [27]:
print commands

[u'docker network create A', u'docker network create C', u'docker network create B', u'docker network create E', u'docker network create D', u'docker network create G', u'docker network create F', u'docker network create H', u'docker run -d --privileged --name host -p 8080:80 --network=A quagga', u'docker run -d --privileged --name server -p 8081:80 --network=B quagga', u'docker run -d --privileged --name rip2 -p 8082:80 --network=B quagga', u'docker run -d --privileged --name rip1 -p 8083:80 --network=H quagga', u'docker run -d --privileged --name br -p 8084:80 --network=G quagga', u'docker run -d --privileged --name ospf1 -p 8085:80 --network=C quagga', u'docker run -d --privileged --name ospf3 -p 8086:80 --network=A quagga', u'docker run -d --privileged --name ospf2 -p 8087:80 --network=D quagga', u'docker network connect H rip2', u'docker network connect G rip1', u'docker network connect F br', u'docker network connect E ospf1', u'docker network connect F ospf1', u'docker network c

In [28]:
print startup_commands

['docker exec -d -ti --privileged=true host ifconfig eth0 1.0.0.2/24 up', 'docker exec -d -ti --privileged=true host route add default gw 1.0.0.1', 'docker exec -d -ti --privileged=true server ifconfig eth0 2.0.0.2/24 up', 'docker exec -d -ti --privileged=true server route add default gw 2.0.0.1', 'docker exec -d -ti --privileged=true server /etc/init.d/apache2 start', 'docker exec -d -ti --privileged=true rip2 ifconfig eth0 2.0.0.1/24 up', 'docker exec -d -ti --privileged=true rip2 ifconfig eth1 11.0.0.2/30 up', 'docker exec -d -ti --privileged=true rip2 /etc/init.d/zebra start', 'docker exec -d -ti --privileged=true rip1 ifconfig eth0 11.0.0.1/30', 'docker exec -d -ti --privileged=true rip1 ifconfig eth1 11.1.0.2/30', 'docker exec -d -ti --privileged=true rip1 /etc/init.d/zebra start', 'docker exec -d -ti --privileged=true br ifconfig eth0 11.1.0.1/30 up', 'docker exec -d -ti --privileged=true br ifconfig eth1 10.1.0.1/30 up', 'docker exec -d -ti --privileged=true br /etc/init.d/zebr

In [29]:
print exec_commands

['docker exec -e TERM=debian -ti --privileged=true host bash  -c "echo -e \'\\033]0;host\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true server bash  -c "echo -e \'\\033]0;server\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true rip2 bash  -c "echo -e \'\\033]0;rip2\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true rip1 bash  -c "echo -e \'\\033]0;rip1\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true br bash  -c "echo -e \'\\033]0;br\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true ospf1 bash  -c "echo -e \'\\033]0;ospf1\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true ospf3 bash  -c "echo -e \'\\033]0;ospf3\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true ospf2 bash  -c "echo -e \'\\033]0;ospf2\\007\'; bash"']


# Run command to create lab

In [30]:
import subprocess
def run_command_detatched(cmd_line):
    p = subprocess.Popen(cmd_line, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    out = p.communicate()[0]
    return out

In [31]:
if DEBUG: print run_command_detatched('ping www.google.com & ping www.google.com')


Esecuzione di Ping www.google.com [172.217.20.196] con 32 byte di dati:
Risposta da 172.217.20.196: byte=32 durata=42ms TTL=50
Risposta da 172.217.20.196: byte=32 durata=44ms TTL=50
Risposta da 172.217.20.196: byte=32 durata=43ms TTL=50
Risposta da 172.217.20.196: byte=32 durata=41ms TTL=50

Statistiche Ping per 172.217.20.196:
    Pacchetti: Trasmessi = 4, Ricevuti = 4, 
    Persi = 0 (0% persi),
Tempo approssimativo percorsi andata/ritorno in millisecondi:
    Minimo = 41ms, Massimo =  44ms, Medio =  42ms

Esecuzione di Ping www.google.com [172.217.20.196] con 32 byte di dati:
Risposta da 172.217.20.196: byte=32 durata=39ms TTL=50
Risposta da 172.217.20.196: byte=32 durata=61ms TTL=50
Risposta da 172.217.20.196: byte=32 durata=40ms TTL=50
Risposta da 172.217.20.196: byte=32 durata=40ms TTL=50

Statistiche Ping per 172.217.20.196:
    Pacchetti: Trasmessi = 4, Ricevuti = 4, 
    Persi = 0 (0% persi),
Tempo approssimativo percorsi andata/ritorno in millisecondi:
    Minimo = 39ms, Mas

In [32]:
separator_unix = ' ; '
separator_windows = ' & '

open_bash_separator = ''

if platform == "Windows":
    open_bash_separator = separator_windows
elif platform == "MAC OS X":
    open_bash_separator = separator_unix
else: #linux or linux2
    open_bash_separator = separator_unix

In [33]:
lab_create_command_string = ''
for command in commands:
    lab_create_command_string += command + open_bash_separator
lab_create_command_string = lab_create_command_string[:len(lab_create_command_string)-2]
if DEBUG: print lab_create_command_string

docker network create A & docker network create C & docker network create B & docker network create E & docker network create D & docker network create G & docker network create F & docker network create H & docker run -d --privileged --name host -p 8080:80 --network=A quagga & docker run -d --privileged --name server -p 8081:80 --network=B quagga & docker run -d --privileged --name rip2 -p 8082:80 --network=B quagga & docker run -d --privileged --name rip1 -p 8083:80 --network=H quagga & docker run -d --privileged --name br -p 8084:80 --network=G quagga & docker run -d --privileged --name ospf1 -p 8085:80 --network=C quagga & docker run -d --privileged --name ospf3 -p 8086:80 --network=A quagga & docker run -d --privileged --name ospf2 -p 8087:80 --network=D quagga & docker network connect H rip2 & docker network connect G rip1 & docker network connect F br & docker network connect E ospf1 & docker network connect F ospf1 & docker network connect D ospf3 & docker network connect C osp

In [34]:
print run_command_detatched(lab_create_command_string)

dc6422fe70d9c0d632ff7e2c7aaee513c39acd4610b4389f4e819bca622d2b8d
92918be80fd38e4e0d2b79b8f7e3f4a36834cd5d85bea61c9d22cedcf41d3c63
563a375c2c9be91104771d6729d54f3c7e127c2cb81f78a6804f3fd346b874e1
7e36f9847babca5747fd62c2494aed636a775bd9bd56aa46637510ac522717a2
52d6fdbb512d75eaa0badf62abff46b693f58903b47c1b56d8e4fdfc0446f0f1
9c54d6b2f1d64c0e55823cb4039f29035f83daeaa3047edc13dba6ed67c54538
d1817f95e0429253a7e9e677694c6efb2aa6a6215976012b02f11a7efc115cb4
2121b8cedfc29aff7a108dbf5cfc56607dc499833201ed93b4e45e85f0876959
33399d96219f2a2fd2302e635d2fd207cdf7d66b5e3b6936bef164a868b05113
501b005c26b3bb6dbc30684266d98225b056cfef5805e62dd0174c4edcc85570
dce00cc6e5e4a11c2af9e6cab2c4c152b162b8c196475d6b7fe3169b1625a035
9b58a9d9b2d9017db553073187f5db76d58b4b5497aaa03aa7c58264cd353e79
08c56d4073be205d2a907afe58e59b13adb7646a806be71dcc40e22dae01f57c
598cf29dc0a0a750fc3c5a65ce7df56b775bca64bdc48e498d0e2084b883641a
264f6a19f0ace2280c7109e8d282d94a717269186c2c3871354b6ce351702824
89e388097145367cd0f655c51

# Run startup commands inside lab and attach to lab

In [35]:
os.system('start /wait cmd /c "ping www.google.com & cmd.exe"')

0

In [36]:
os.system(LINUX_TERMINAL_TYPE + ' -e \'bash -c "ping www.google.com; exec bash"\'')

1

In [37]:
os.system('Terminal.app -c "ping www.google.com; exec Terminal.app"') # not tested yet

1

In [38]:
open_bash_template_linux = LINUX_TERMINAL_TYPE + ' -e \'bash -c "{commands}"\''
open_bash_template_windows = 'start cmd /k "{commands}"'
open_bash_template_osx = 'Terminal.app -e \'bash -c "{commands}"\''  # not tested yet

open_bash_template = ''

if platform == "Windows":
    open_bash_template = open_bash_template_windows
elif platform == "MAC OS X":
    open_bash_template = open_bash_template_osx
else: #linux or linux2
    open_bash_template = open_bash_template_linux

In [39]:
for startup_command in startup_commands:
    print run_command_detatched(startup_command)





Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \"/etc/init.d/apache2\": stat /etc/init.d/apache2: no such file or directory"



Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \"/etc/init.d/zebra\": stat /etc/init.d/zebra: no such file or directory"



Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \"/etc/init.d/zebra\": stat /etc/init.d/zebra: no such file or directory"



Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \"/etc/init.d/zebra\": stat /etc/init.d/zebra: no such file or directory"




Error response from daemon: rpc error: code = 2 desc = oci runtime error: e

In [40]:
command_string = ''
for command in exec_commands:
    command_string += open_bash_template.replace('{commands}', command) + open_bash_separator
command_string = command_string[:len(command_string)-2]
if DEBUG: print command_string

start cmd /k "docker exec -e TERM=debian -ti --privileged=true host bash  -c "echo -e '\033]0;host\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true server bash  -c "echo -e '\033]0;server\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip2 bash  -c "echo -e '\033]0;rip2\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip1 bash  -c "echo -e '\033]0;rip1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true br bash  -c "echo -e '\033]0;br\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf1 bash  -c "echo -e '\033]0;ospf1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf3 bash  -c "echo -e '\033]0;ospf3\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf2 bash  -c "echo -e '\033]0;ospf2\007'; bash"" 


In [41]:
print(open_bash_template.replace('{commands}', command_string))

start cmd /k "start cmd /k "docker exec -e TERM=debian -ti --privileged=true host bash  -c "echo -e '\033]0;host\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true server bash  -c "echo -e '\033]0;server\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip2 bash  -c "echo -e '\033]0;rip2\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip1 bash  -c "echo -e '\033]0;rip1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true br bash  -c "echo -e '\033]0;br\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf1 bash  -c "echo -e '\033]0;ospf1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf3 bash  -c "echo -e '\033]0;ospf3\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf2 bash  -c "echo -e '\033]0;ospf2\007'; bash"" "


In [42]:
os.system(open_bash_template.replace('{commands}', command_string))

0